In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch
import pandas as pd
import os
import deepspeed


In [ ]:
model_name_or_path = "/mnt/sdc/yuzhao/model/llm/llama/llama2-7b-chat-hf"
# To use a different branch, change revision
# For example: revision="gptq-4bit-64g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, 
                                          trust_remote_code=True,
                                          use_fast=True)

In [ ]:
print(model.config.model_type)

In [ ]:
prompt = '''
claim: 1. A peptide having the amino acid sequence of the SEQ ID NO: 1 and having anti-inflammatory activity and inhibitory effects of transglutaminase and phospholipase A2.
claim: 2. A peptide having the amino acid sequence of the SEQ ID NO:2 and having anti-inflammatory activity and inhibitory effects of transglutaminase and phospholipase A2.
claim: 3. A peptide having the amino acid sequence of the SEQ ID NO:3 and having anti-inflammatory activity and inhibitory effects of transglutaminase and phospholipase A2.
claim: 4. A derivative of the peptide of any one of claims 1, 2 and 3, wherein at least one side chain amine group of said peptide is acylated or arylated, or at least one hydroxyl group is esterified to an alkyl group or an aryl group, and which derivative exhibits anit-inflammatory activity and inhibitory effects of transglutaminase and phospholipase A2.
claim: 5. An analog of the peptide of any one of claims 1, 2 and 3, wherein said analog contains at least one amino acid mimic, which serves to reduce proteolytic cleavage of the analog as compared to the peptide, and which analog exhibits anti-inflammatory activity and inhibitory effects of transglutaminase and phospholipase A2.
claim: 6. A pharmaceutical composition for anti-inflammation and inhibitory effects of transglutaminase and phospholipase A2, comprising the peptide, peptide derivative or peptide analog of any one of claims 1 to 5 and pharmaceutically acceptable carrier.
claim: 7. A method for inhibiting transglutaminase and inflammatory response that is associated or caused by an inflammatory disease, comprising the step of exposing humans or animals to an effective amount of the peptide, peptide derivative or peptide analog of any one of claims 1 to 5.
claim: 8. The method of claim 7, wherein said inflammatory disease comprises autoimmune diseases such as ulcerative colitis, rheumatoid arthritis, scleroderma, inflammatory lung disease, celiac disease, systemic lupus, myasthenia gravis and diabetes; various allergic or immune diseases such as skin allergy, pimples or trauma.
claim: 9. The method of claim 7, wherein said inflammatory response comprises degenerative diseases such as Parkinson's disease, Huntington's disease and Alzheimer's disease; painful diseases or nervous diseases.

Given the above texts, what are the functions of the SEQ ID NO(peptide sequences) mentioned in the text? Take a deep breath and Let's work this out in a step-by-step way to be sure we have the right answer.The output format should be Markdown, the first column should be 'claim NO', the second column should be SEQ ID NO, and the third column should be functions.If it is a combination peptide, the SEQ ID NO is a list of the sequence of the combination peptides.If the SEQ ID NO is indirectly mentioned, return No.
Please do not return anything other than the answer.
'''
prompt_template=f'''
[INST] 
<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}
[/INST]
'''


input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0, top_p=1, max_new_tokens=1024)
print(output.shape)  # torch.Size([1, 1650])
outputs = output.tolist()[0][len(input_ids[0]):]
print("*** Generate:")
print(tokenizer.decode(outputs))